# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mukalla,14.5425,49.1242,24.91,80,86,0.40,YE,1706461622
1,1,khabary,53.6259,79.5345,-22.60,100,96,1.84,RU,1706461622
2,2,papatowai,-46.5619,169.4708,12.43,87,90,1.64,NZ,1706461623
3,3,waitangi,-43.9535,-176.5597,17.68,97,100,4.47,NZ,1706461623
4,4,kavaratti,10.5669,72.6420,27.53,65,29,1.38,IN,1706461623


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City"],
    colormap="plasma",
    title="City Map with Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    size="Humidity",
    frame_width=600,
    frame_height=400,
    tiles="OSM",
    geo=True
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
fit_city_data_df = city_data_df[
    (city_data_df["Max Temp"] > 27) & (city_data_df["Max Temp"] < 35) &
    (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 15)
]

# Drop any rows with null values
drop_city_data = fit_city_data_df.dropna()

# Display sample data
drop_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,vallenar,-28.5708,-70.7581,30.10,35,0,4.40,CL,1706461396
134,134,kombissiri,12.0656,-1.3375,31.73,10,3,3.99,BF,1706461655
162,162,benguela,-12.5763,13.4055,28.02,73,14,3.23,AO,1706461662
181,181,estebania,18.4577,-70.6435,29.61,48,5,2.77,DO,1706461667
189,189,freetown,8.4840,-13.2299,28.49,62,9,3.97,SL,1706461669
191,191,juan de ayolas,-27.4000,-56.9000,30.86,39,0,1.45,PY,1706461669
314,314,chicoana,-25.1009,-65.5331,28.92,52,11,1.81,AR,1706461696
349,349,banyo,6.7500,11.8167,27.78,22,5,1.30,CM,1706461705
365,365,bouna,9.2667,-3.0000,34.92,9,5,2.80,CI,1706461710
400,400,fada,9.5667,8.3000,30.72,15,2,0.82,NG,1706461718


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = drop_city_data.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,115,vallenar,-28.5708,-70.7581,30.10,35,0,4.40,CL,1706461396,
134,134,kombissiri,12.0656,-1.3375,31.73,10,3,3.99,BF,1706461655,
162,162,benguela,-12.5763,13.4055,28.02,73,14,3.23,AO,1706461662,
181,181,estebania,18.4577,-70.6435,29.61,48,5,2.77,DO,1706461667,
189,189,freetown,8.4840,-13.2299,28.49,62,9,3.97,SL,1706461669,
191,191,juan de ayolas,-27.4000,-56.9000,30.86,39,0,1.45,PY,1706461669,
314,314,chicoana,-25.1009,-65.5331,28.92,52,11,1.81,AR,1706461696,
349,349,banyo,6.7500,11.8167,27.78,22,5,1.30,CM,1706461705,
365,365,bouna,9.2667,-3.0000,34.92,9,5,2.80,CI,1706461710,
400,400,fada,9.5667,8.3000,30.72,15,2,0.82,NG,1706461718,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "tourism,accommodation",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
vallenar - nearest hotel: No hotel found
kombissiri - nearest hotel: Hotel MITIBKIETA
benguela - nearest hotel: Hotel Moibela
estebania - nearest hotel: la mami 
freetown - nearest hotel: central guesthouse
juan de ayolas - nearest hotel: Hotel Nacional de Turismo Ayolas
chicoana - nearest hotel: Hostal Los Faroles
banyo - nearest hotel: Hôtel Amin
bouna - nearest hotel: Hôtel Éléphant  Bouna
fada - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel Porto La Belle Group
felipe carrillo puerto - nearest hotel: Hotel María José
wamba - nearest hotel: No hotel found
tan-tan - nearest hotel: Sable d´or
takoradi - nearest hotel: Hillcrest


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,115,vallenar,-28.5708,-70.7581,30.10,35,0,4.40,CL,1706461396,No hotel found
134,134,kombissiri,12.0656,-1.3375,31.73,10,3,3.99,BF,1706461655,Hotel MITIBKIETA
162,162,benguela,-12.5763,13.4055,28.02,73,14,3.23,AO,1706461662,Hotel Moibela
181,181,estebania,18.4577,-70.6435,29.61,48,5,2.77,DO,1706461667,la mami
189,189,freetown,8.4840,-13.2299,28.49,62,9,3.97,SL,1706461669,central guesthouse
191,191,juan de ayolas,-27.4000,-56.9000,30.86,39,0,1.45,PY,1706461669,Hotel Nacional de Turismo Ayolas
314,314,chicoana,-25.1009,-65.5331,28.92,52,11,1.81,AR,1706461696,Hostal Los Faroles
349,349,banyo,6.7500,11.8167,27.78,22,5,1.30,CM,1706461705,Hôtel Amin
365,365,bouna,9.2667,-3.0000,34.92,9,5,2.80,CI,1706461710,Hôtel Éléphant Bouna
400,400,fada,9.5667,8.3000,30.72,15,2,0.82,NG,1706461718,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
hotel_city_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    colormap="plasma",
    title="City Map with Humidity and Hotel Information",
    xlabel="Longitude",
    ylabel="Latitude",
    size="Humidity",
    frame_width=600,
    frame_height=400,
    tiles="OSM",
    geo=True
)

# Display the map
hotel_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)